# CCLE SNP Fingerprinting Pipeline
Author: William Colgan (wcolgan@broadinstitute.org)

In [ ]:
# Load packages

import pandas as pd
import numpy as np
import dalmatian as dm
import subprocess
from taigapy import TaigaClient
from functools import reduce

tc = TaigaClient()

In [ ]:
# Pipeline parameters

# CSV file of bams to fingerprint with columns: id, bam_filepath, bai_filepath
bams_sheet = "/Users/wcolgan/Desktop/bams_to_fingerprint.csv" 

# Local directory to save intermediate files to
working_dir = "/Users/wcolgan/Desktop/fingerprint/"

# GC storage bucket containing fingerprints
fingerprints_dir = "gs://fc-secure-6b6a3e1a-6fb8-4d30-b0df-a359e6c5d6e6/fingerprints"

# GC storage bucket containing lists for vcf files
vcf_list_dir = "gs://fc-secure-6b6a3e1a-6fb8-4d30-b0df-a359e6c5d6e6/vcf_lists"

# Batch sise for crosscheck_vcf. If more than 200 bams are being run this should be decreased
crosscheck_batch_size = 500

## Generate Fingerprint VCFs

Here we use Dalmatian to run the `fingerprint_bam_with_liftover` workflow on Terra. 
This workflow calls Picard ExtractFingerprint to generate a fingerprint VCF and then 
calls Picard LiftoverVcf to covert this vcf to hg38. To fingerprint hg38 bam files just run `fingerprint_bam` instead.

In [ ]:
bams = pd.read_csv(bams_sheet)
bams

In [ ]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/CCLE_SNP_QC")

In [ ]:
# Upload sample sheet
samples_df = bams[["bam_filepath","bai_filepath"]]
samples_df[["sample_id"]] = bams[["id"]]
samples_df[["participant_id"]] = bams[["id"]]
samples_df = samples_df.set_index('sample_id')
wm.upload_samples(samples_df, add_participant_samples=True)
wm.update_sample_set('all_samples', samples_df.index)

In [ ]:
# Submit jobs 
wm.create_submission("fingerprint_bam_with_liftover",'all_samples','sample_set',expression='this.samples')
#wm.create_submission("fingerprint_bam",'all_samples','sample_set',expression='this.samples')

In [ ]:
# Monitor jobs
wm.get_submission_status()

## Move Fingerprint VCFs

Here we move the fingerprint vcf files into `fingerprints_dir`

In [ ]:
samples_df = wm.get_samples()
cmd = 'echo -e "{}" | gsutil -m cp -I {}'.format('\n'.join(samples_df["fingerprint_vcf"]), fingerprints_dir)
subprocess.check_call(cmd, shell=True, executable='/bin/bash')

## Crosscheck Fingerprint VCFs

Here we use Dalmation to run the `crosscheck_vcfs` workflow on Terra. This workflow calls Picard CrosscheckFingerprints to compare the new fingerprint vcfs to batches of existing fingerprint vcfs in `fingerprints_dir`

In [ ]:
# Create batch files listing all vcfs in fingerprints dir and upload to bucket
cmd = "gsutil ls "+fingerprints_dir
out, err = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, executable='/bin/bash').communicate()
vcf_list = str(out).split("\\n")
vcf_list = vcf_list[1:len(vcf_list)-1]
num = 0
batches = [] 
for i in range(0, len(vcf_list), crosscheck_batch_size):
    f = open(working_dir + "vcf_batch_"+str(num), 'w')
    f.write("\n".join(vcf_list[i:i + crosscheck_batch_size]))
    f.close()
    batches += ["vcf_batch_"+str(num)]
    num += 1
cmd = 'echo -e "{}" | gsutil -m cp -I {}'.format(working_dir+('\n'+working_dir).join(batches),vcf_list_dir)
subprocess.check_call(cmd, shell=True, executable='/bin/bash')

In [ ]:
# Create list with new vcfs and upload to bucket
f = open(working_dir + "new_vcfs", 'w')
f.write(fingerprints_dir+'/'+('.vcf\n'+fingerprints_dir+'/').join(bams["id"])+'.vcf')
f.close()
cmd = 'gsutil cp '+working_dir+'new_vcfs '+vcf_list_dir
subprocess.check_call(cmd, shell=True, executable='/bin/bash')

In [ ]:
# Upload sample sheet
samples_df = pd.DataFrame(data={"sample_id" : batches, "participant_id" : batches, \
                    "vcf_input_file" : [vcf_list_dir+'/new_vcfs']*len(batches), \
                   "vcf_second_input_file" : [vcf_list_dir+'/'+x for x in batches]})
samples_df = samples_df.set_index('sample_id')
wm.upload_samples(samples_df, add_participant_samples=True)
wm.update_sample_set('all_samples', samples_df.index)

In [ ]:
# Submit jobs
wm.create_submission("crosscheck_vcfs",'all_samples','sample_set',expression='this.samples')

In [ ]:
# Monitor jobs
wm.get_submission_status()

## Update LOD matrix

Here we update the fingerprint LOD matrix on taiga with the new fingerprints

In [ ]:
# Download crosscheck results
samples_df = wm.get_samples()
cmd = 'echo -e "{}" | gsutil -m cp -I {}'.format('\n'.join(samples_df.loc[batches,"crosscheck_out"]),working_dir)
subprocess.check_call(cmd, shell=True, executable='/bin/bash')

In [ ]:
# Generate matrix with LOD score for new fingerprint vcfs
new_lod_list = []
for batch in batches:
    df = pd.read_csv(working_dir+batch+"_crosscheck",sep='\t',comment='#')
    lod_mat = df.pivot(index = "LEFT_SAMPLE",columns="RIGHT_SAMPLE",values = "LOD_SCORE")
    new_lod_list += [lod_mat]
new_lod_mat = reduce(lambda x, y: pd.merge(x, y, left_index = True, right_index = True), new_lod_list)
new_lod_mat.index.name = None
new_lod_mat

In [ ]:
lod_mat =  tc.get(name='ccle-bam-fingerprints-6f30',file='fingerprint_lod_matrix')

In [ ]:
# Update LOD matrix
new_ids = set(new_lod_mat.index)
old_ids = set(lod_mat.index).difference(new_ids)
updated_lod_mat = pd.concat((lod_mat.loc[old_ids,old_ids],new_lod_mat.loc[new_ids,old_ids]), axis=0)
updated_lod_mat = pd.concat((updated_lod_mat.loc[new_ids.union(old_ids),old_ids], \
                             new_lod_mat.transpose().loc[new_ids.union(old_ids,new_ids)]), axis=1)
updated_lod_mat

In [ ]:
# Upload updated LOD matrix to Tiaga
updated_lod_mat.to_csv(working_dir+'fingerprint_lod_matrix.csv')
tc.update_dataset(dataset_permaname = "ccle-bam-fingerprints-6f30",
                  changes_description="New bam fingerprints added",
                  upload_file_path_dict=[
                    {
                        "path": working_dir+'fingerprint_lod_matrix.csv',
                        "name": "fingerprint_lod_matrix",
                        "format": "NumericMatrixCSV",
                        "encoding": "utf-8"
                    }
                 ],
                 add_all_existing_files=True)